In [ ]:
#Install dependencies

!pip install -q streamlit crewai crewai_tools reportlab python-docx pyngrok PyPDF2

In [13]:
%%writefile app.py
import os
import streamlit as st
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, FileReadTool, MDXSearchTool
import io
import docx
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
import traceback
import tempfile
import PyPDF2
from docx import Document


def create_docx(content, filename):
    """Create a Word document from text content"""
    doc = docx.Document()
    for line in content.split('\n'):
        doc.add_paragraph(line)

    doc_io = io.BytesIO()
    doc.save(doc_io)
    doc_io.seek(0)
    return doc_io


def setup_crew(job_posting_url, github_url, personal_writeup, openai_api_key, serper_api_key, gpt_model, resume_file_path):
    """Set up CrewAI agents and tasks"""
    if not openai_api_key:
        raise ValueError("OpenAI API Key is missing. Please provide a valid API key.")
    if not serper_api_key:
        raise ValueError("Serper API Key is missing. Please provide a valid API key.")

    try:
        os.environ["OPENAI_API_KEY"] = openai_api_key
        os.environ["SERPER_API_KEY"] = serper_api_key

        search_tool = SerperDevTool()
        scrape_tool = ScrapeWebsiteTool()

        try:
            file_read_tool = FileReadTool(file_path=resume_file_path)
            semantic_search_tool = MDXSearchTool(file_path=resume_file_path)
        except Exception as tool_error:
            st.warning(f"Could not initialize file-related tools: {tool_error}")
            file_read_tool = None
            semantic_search_tool = None

        researcher = Agent(
            role="Tech Job Researcher",
            goal="Make sure to do amazing analysis on job posting to help job applicants",
            tools=[scrape_tool, search_tool],
            verbose=True,
            backstory="As a Job Researcher, your prowess in navigating and extracting critical information from job postings is unmatched.",
            llm_config={
                "temperature": 0.7,
                "model_name": gpt_model,
                "api_key": openai_api_key
            }
        )

        profiler = Agent(
            role="Personal Profiler for Engineers",
            goal="Do incredible research on job applicants to help them stand out in the job market",
            tools=[scrape_tool, search_tool, file_read_tool, semantic_search_tool],
            verbose=True,
            backstory="Equipped with analytical prowess, you dissect and synthesize information from diverse sources to craft comprehensive personal and professional profiles.",
            llm_config={
                "temperature": 0.7,
                "model_name": gpt_model,
                "api_key": openai_api_key
            }
        )

        resume_strategist = Agent(
            role="Resume Strategist for Engineers",
            goal="Find all the best ways to make a resume stand out in the job market",
            tools=[scrape_tool, search_tool, file_read_tool, semantic_search_tool],
            verbose=True,
            backstory="With a strategic mind and an eye for detail, you excel at refining resumes to highlight the most relevant skills and experiences.",
            llm_config={
                "temperature": 0.7,
                "model_name": gpt_model,
                "api_key": openai_api_key
            }
        )

        interview_preparer = Agent(
            role="Engineering Interview Preparer",
            goal="Create interview questions and talking points based on the resume and job requirements",
            tools=[scrape_tool, search_tool, file_read_tool, semantic_search_tool],
            verbose=True,
            backstory="Your role is crucial in anticipating the dynamics of interviews, formulating key questions and talking points.",
            llm_config={
                "temperature": 0.7,
                "model_name": gpt_model,
                "api_key": openai_api_key
            }
        )

        research_task = Task(
            description=f"Analyze the job posting URL {job_posting_url} to extract key skills, experiences, and qualifications required.",
            expected_output="A structured list of job requirements, including necessary skills, qualifications, and experiences.",
            agent=researcher
        )

        profile_task = Task(
            description=f"Compile a detailed personal and professional profile using GitHub URL {github_url} and personal write-up.",
            expected_output="A comprehensive profile document that includes skills, project experiences, contributions, interests, and communication style.",
            agent=profiler
        )

        resume_strategy_task = Task(
            description="Using the profile and job requirements, tailor the resume to highlight the most relevant areas.",
            expected_output="An updated resume that effectively highlights the candidate's qualifications and experiences.",
            agent=resume_strategist,
            output_file="tailored_resume.md",
            context=[research_task, profile_task]
        )

        interview_preparation_task = Task(
            description="Create potential interview questions and talking points based on the tailored resume and job requirements.",
            expected_output="A document containing key questions and talking points for the initial interview.",
            agent=interview_preparer,
            output_file="interview_materials.md",
            context=[research_task, profile_task, resume_strategy_task]
        )

        job_application_crew = Crew(
            agents=[researcher, profiler, resume_strategist, interview_preparer],
            tasks=[research_task, profile_task, resume_strategy_task, interview_preparation_task],
            verbose=True
        )

        job_application_inputs = {
            'job_posting_url': job_posting_url,
            'github_url': github_url,
            'personal_writeup': personal_writeup
        }

        result = job_application_crew.kickoff(inputs=job_application_inputs)

        return result

    except Exception as e:
        st.error(f"Error setting up CrewAI: {str(e)}")
        st.error(f"Detailed Traceback:\n{traceback.format_exc()}")
        raise


def main():
    st.title("Resume Tailoring AI Assistant 📄✨")

    st.sidebar.header("API Configuration")
    openai_api_key = st.sidebar.text_input("OpenAI API Key", type="password",
                                           help="Your OpenAI API key from platform.openai.com")
    serper_api_key = st.sidebar.text_input("Serper API Key", type="password",
                                           help="Your Serper API key from serper.dev")

    gpt_models = ["gpt-3.5-turbo", "gpt-4", "gpt-4o"]
    gpt_model = st.sidebar.selectbox("Select GPT Model", options=gpt_models)

    st.sidebar.header("Job Application Details")
    job_posting_url = st.sidebar.text_input("Job Posting URL")
    github_url = st.sidebar.text_input("GitHub Profile URL")
    personal_writeup = st.sidebar.text_area("Personal Professional Summary")
    resume_file = st.sidebar.file_uploader("Upload Resume (Markdown/Text)", type=['md', 'txt', 'pdf', 'docx'])

    if st.sidebar.button("Generate Tailored Materials"):
        if not all([openai_api_key, serper_api_key, job_posting_url, github_url, personal_writeup, resume_file]):
            st.error("Please fill out all required fields.")
            return

        with st.spinner('Processing uploaded resume...'):
            temp_file_path = None
            try:
                if resume_file.type == "application/pdf":
                    pdf_reader = PyPDF2.PdfReader(resume_file)
                    resume_content = "\n".join(page.extract_text() or '' for page in pdf_reader.pages)
                elif resume_file.type == "application/vnd.openxmlformats-officedocument.wordprocessingml.document":
                    doc = Document(resume_file)
                    resume_content = "\n".join(paragraph.text for paragraph in doc.paragraphs)
                elif resume_file.type in ["text/plain", "text/markdown"]:
                    resume_content = resume_file.getvalue().decode('utf-8', errors='replace')
                else:
                    st.error("Unsupported file type.")
                    return

                with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix='.md') as temp_file:
                    temp_file.write(resume_content)
                    temp_file_path = temp_file.name

                final_result = setup_crew(
                    job_posting_url, github_url, personal_writeup,
                    openai_api_key, serper_api_key, gpt_model, temp_file_path
                )

                with open("./tailored_resume.md", "r", encoding="utf-8") as file:
                    markdown_resume_content = file.read()
                docx_resume = create_docx(markdown_resume_content, 'tailored_resume.docx')

                with open("./interview_materials.md", "r", encoding="utf-8") as file:
                    markdown_interview_content = file.read()
                docx_interview = create_docx(markdown_interview_content, 'interview_materials.docx')

                st.session_state["docx_resume"] = docx_resume
                st.session_state["docx_interview"] = docx_interview

                st.success("Materials generated successfully!")
            except Exception as e:
                st.error(f"Error occurred: {e}")
            finally:
                if temp_file_path:
                    os.unlink(temp_file_path)
    # Display download buttons if content exists
    if "docx_resume" in st.session_state and "docx_interview" in st.session_state:
        st.subheader("Download Your Tailored Materials")
        col1, col2 = st.columns(2)
        with col1:
            st.download_button(
                label="Download Word Resume",
                data=st.session_state["docx_resume"],
                file_name="tailored_resume.docx",
                mime="application/vnd.openxmlformats-officedocument.wordprocessingml.document",
                key="resume_download"
            )
        with col2:
            st.download_button(
                label="Download Word Interview Materials",
                data=st.session_state["docx_interview"],
                file_name="interview_materials.docx",
                mime="application/vnd.openxmlformats-officedocument.wordprocessingml.document",
                key="interview_materials_download"
            )


if __name__ == "__main__":
    main()

Overwriting app.py


In [14]:
from pyngrok import ngrok

# Kill any previous Streamlit instances
!kill -9 $(lsof -t -i:8501)

# Start a new ngrok tunnel
#public_url = ngrok.connect(port='8501')
#print(f"Public URL for the app: {public_url}")

# Configure ngrok with your authtoken
ngrok.set_auth_token("<YOUR TOKEN")  # Replace with your actual token


public_url = ngrok.connect(8501, proto="http")  # Specify proto="http"
print(f"Public URL: {public_url}")

Public URL: NgrokTunnel: "https://ba8d-34-139-79-236.ngrok-free.app" -> "http://localhost:8501"


In [15]:
!streamlit run app.py &>/content/logs.txt &